In [16]:
import os
import numpy as np
from obspy import read
from obspy.signal.filter import bandpass
from scipy.signal import stft

# Parameters
input_folder = "./data/raw"  # Folder with raw seismic data (.mseed files)
output_folder = "./data/processed_data"  # Folder to save processed results
os.makedirs(output_folder, exist_ok=True)

target_sampling_rate = 10  # Hz
filter_min = 1  # Hz
filter_max = 4  # Hz
window_duration = 3600  # seconds (1 hour)
overlap = 0.5  # Overlap between windows (50%)

# Calculate parameters

nperseg = 190  # Ensures 96 frequency bins
noverlap = int(nperseg * (1 - 128 / (28800 / nperseg)))  # Adjust for 128 time bins


# Initialize list for processed data
processed_data = []

# Helper function to process a single file
def process_file(filepath):
    # Read the seismic data
    stream = read(filepath)
    trace = stream[0]
    
    sampling_rate = trace.stats.sampling_rate
    
    # Demean and detrend
    trace.detrend(type="demean")
    trace.detrend("linear")
    
    # Resample to target sampling rate
    trace.resample(sampling_rate=target_sampling_rate)
    
    # Bandpass filter
    trace.data = bandpass(trace.data, filter_min, filter_max, trace.stats.sampling_rate, corners=4, zerophase=True)
    
    # STFT computation
    f, t, Zxx = stft(
        trace.data,
        fs=sampling_rate,
        nperseg=nperseg,
        noverlap=noverlap,
    )
    
    # Ensure the output has the correct shape
    stft_magnitude = np.abs(Zxx)
    if stft_magnitude.shape != (96, 128):
        raise ValueError(f"STFT output shape mismatch: {stft_magnitude.shape}")
    
    return stft_magnitude


In [17]:

# Process all files in the folder
for filename in os.listdir(input_folder):
    if filename.endswith(".mseed"):
        filepath = os.path.join(input_folder, filename)
        print(f"Processing file: {filepath}")
        stft_magnitude = process_file(filepath)
        display(stft_magnitude.shape)
        processed_data.append(stft_magnitude)  # Append processed file to the list

# Convert list to a single NumPy array
processed_data_array = np.array(processed_data, dtype=object)
print(f"Final array shape: {processed_data_array.shape}")

# Save the array
output_array_path = os.path.join(output_folder, "processed_seismic_data.npy")
np.save(output_array_path, processed_data_array)
print(f"Combined data saved to: {output_array_path}")


Processing file: ./data/raw/20210430_9fnuph.mseed


ValueError: STFT output shape mismatch: (96, 5905)